<a href="https://colab.research.google.com/github/JoaoMarcosSSPereira/landingpageJMANALYTICS/blob/main/Calculadora_Interativa_de_Renda_Freelancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Função de cálculo da renda
def calcular_renda(nivel_tecnico, infra_produtiva, horas_dia, dias_uteis, meta_renda):
    tempo_por_job = 3 # Tempo médio em horas por job
    valor_medio_job = nivel_tecnico * 10 + 20 # Valor médio por job baseado no nível técnico
    horas_total = horas_dia * dias_uteis * (1 + infra_produtiva / 100) # Horas totais disponíveis, ajustadas pela produtividade
    jobs_possiveis = int(horas_total // tempo_por_job) # Número de jobs que podem ser feitos
    renda_gerada = jobs_possiveis * valor_medio_job # Renda total gerada
    gap = meta_renda - renda_gerada # Diferença entre a meta e a renda gerada
    return horas_total, jobs_possiveis, valor_medio_job, renda_gerada, gap

# Widgets interativos para os parâmetros de entrada
nivel_tecnico_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='Nível Técnico')
infra_produtiva_slider = widgets.IntSlider(value=30, min=0, max=100, step=5, description='Produtividade (%)')
horas_dia_slider = widgets.IntSlider(value=5, min=1, max=8, step=1, description='Horas/dia')
dias_uteis_slider = widgets.IntSlider(value=20, min=10, max=25, step=1, description='Dias úteis')
meta_renda_input = widgets.FloatText(value=600, description='Meta (R$)')

# Área de saída para exibir os resultados e o gráfico
output = widgets.Output()

# Função para atualizar o dashboard com base nas mudanças dos widgets
def atualizar_dashboard(change=None):
    with output:
        clear_output(wait=True) # Limpa a saída anterior antes de exibir novos resultados
        nt = nivel_tecnico_slider.value
        ip = infra_produtiva_slider.value
        hd = horas_dia_slider.value
        du = dias_uteis_slider.value
        mr = meta_renda_input.value

        # Calcula a renda com os valores atuais dos widgets
        ht, jp, vj, rg, gap = calcular_renda(nt, ip, hd, du, mr)

        # Formata os resultados da simulação em HTML para uma apresentação mais bonita
        summary_html = f"""
        <div style="border: 1px solid #444; padding: 15px; border-radius: 8px; background-color: #2a2a2a; color: #e0e0e0; font-family: sans-serif;">
            <h3 style="color: #66ccff;">🔍 Simulação Freelancer</h3>
            <hr style="border-color: #555;">
            <p><strong>Total de horas disponíveis (com infra):</strong> <span style="color: #aaffaa;">{ht:.1f} h</span></p>
            <p><strong>Quantidade de jobs possíveis:</strong> <span style="color: #aaffaa;">{jp}</span></p>
            <p><strong>Valor médio por job:</strong> <span style="color: #aaffaa;">R$ {vj:.2f}</span></p>
            <p><strong>Renda estimada:</strong> <span style="color: #aaffaa;">R$ {rg:.2f}</span></p>
            <p><strong>Meta de renda:</strong> <span style="color: #aaffaa;">R$ {mr:.2f}</span></p>
        """
        if gap > 0:
            summary_html += f"""
            <p style="color: #ffaa00;">⚠️ <strong>Você está R$ {gap:.2f} abaixo da meta.</strong></p>
            """
        else:
            summary_html += f"""
            <p style="color: #00ff00;">✅ <strong>Meta de renda ATINGIDA! Excedente: R$ {abs(gap):.2f}</strong></p>
            """
        summary_html += "</div>"

        display(HTML(summary_html)) # Exibe o HTML formatado

        # Geração do gráfico de calor: renda por nível técnico e horas por dia
        niveis = np.arange(1, 11) # Níveis técnicos de 1 a 10
        horas = np.arange(2, 9)   # Horas por dia de 2 a 8
        renda = np.zeros((len(niveis), len(horas))) # Matriz para armazenar a renda

        # Preenche a matriz de renda para o gráfico de calor
        for i, n in enumerate(niveis):
            for j, h in enumerate(horas):
                vj_calc = n * 10 + 20
                ht_calc = h * du * (1 + ip / 100)
                jobs_calc = int(ht_calc // 3)
                renda[i, j] = jobs_calc * vj_calc

        # Cria o gráfico de calor usando Plotly
        fig = go.Figure(data=go.Heatmap(
            z=renda,
            x=horas,
            y=niveis,
            colorscale='Viridis',
            colorbar_title='R$ Estimado'
        ))
        fig.update_layout(
            title='Simulação de Renda por Nível Técnico e Horas por Dia',
            xaxis_title='Horas por Dia',
            yaxis_title='Nível Técnico',
            template='plotly_dark' # Define um tema escuro para o gráfico
        )
        fig.show() # Exibe o gráfico

# Conecta a função de atualização a cada mudança nos widgets
nivel_tecnico_slider.observe(atualizar_dashboard, names='value')
infra_produtiva_slider.observe(atualizar_dashboard, names='value')
horas_dia_slider.observe(atualizar_dashboard, names='value')
dias_uteis_slider.observe(atualizar_dashboard, names='value')
meta_renda_input.observe(atualizar_dashboard, names='value')

# Empacota todos os widgets e a saída em uma caixa vertical
calculadora_freela = widgets.VBox([
    widgets.HTML("<h2>🧮 Calculadora Interativa de Renda Freelancer</h2>"),
    nivel_tecnico_slider,
    infra_produtiva_slider,
    horas_dia_slider,
    dias_uteis_slider,
    meta_renda_input,
    output
])

# Exibe a calculadora completa
display(calculadora_freela)

# Atualiza o dashboard na inicialização para mostrar os valores padrão
atualizar_dashboard()